In [1]:
import sys
from collections import deque
import numpy as np
import theano
import theano.tensor as T
import lasagne
sys.path.append("..")
from qtable import QAgent, SimpleMaze
from qnn import QAgentNN, wrap_state

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
maze = SimpleMaze()
agent = QAgentNN(dim_state=(1, 1, 2), range_state=((((0, 3),(0, 4)),),),actions=maze.actions,
                 learning_rate=0.01, reward_scaling=100,
                 freeze_period=100,
                 alpha=0.5, gamma=0.5, explore_strategy='fixed_epsilon', epsilon=0.2)

In [4]:
# logging
path = deque()  # path in this episode
episode_reward_rates = []
num_episodes = 0
cum_reward = 0
cum_steps = 0

# repeatedly run episodes
while num_episodes < 6000:
    maze.reset()
    new_state = maze.observe()
    agent.reset(init_state=wrap_state(new_state))

    path.clear()
    path.append(new_state)
    episode_reward = 0
    episode_steps = 0

    # interact and reinforce repeatedly
    while not maze.finished():
        action = agent.act()
        new_state, reward = maze.interact(action)
        agent.reinforce(new_state=wrap_state(new_state), reward=reward)

        path.append(new_state)
        episode_reward += reward
        episode_steps += 1

    cum_steps += episode_steps
    cum_reward += episode_reward
    num_episodes += 1
    episode_reward_rates.append(episode_reward / episode_steps)
    if num_episodes % 100 == 0:
        print num_episodes, 1.0 * cum_reward / cum_steps#, path
win = 50
s = pd.rolling_mean(pd.Series([0]*win+episode_reward_rates), window=win, min_periods=1)
s.plot()
plt.show()

100 33.00330033
200 33.8983050847
300 34.2857142857
400 34.9344978166
500 35.360678925
600 35.3565114909
700 35.2289884248
800 34.6921075455
900 34.9922239502
1000 34.9040139616
1100 34.8763474952
1200 34.8027842227
1300 34.7779561263
1400 34.4657804037
1500 34.137460173
1600 34.3347639485
1700 34.2121151137
1800 33.9302544769
1900 33.8680926916
2000 33.9904826649
2100 33.9202067517
2200 33.8357428484
2300 33.7738619677
2400 33.8839474799
2500 33.8524035206
2600 33.8629851524
2700 33.8685398896
2800 33.836858006
2900 33.7641168937
3000 33.8180588434
3100 33.7690631808
3200 33.8875357408
3300 33.9995878838
3400 33.8072983991
3500 33.8917400988
3600 33.8696020322
3700 33.8332114119
3800 33.8078291815
3900 33.8072122053
4000 33.7581230484
4100 33.7448559671
4200 33.7132766094
4300 33.6410577374
4400 33.6932383797
4500 33.6297735595
4600 33.5179248033
4700 33.5906232133
4800 33.5781741868
4900 33.566241951
5000 33.5773285877
5100 33.5791414274
5200 33.5700451904
5300 33.5570469799
5400 33.

NameError: name 'pd' is not defined